### Mental Health Risk Assessment UI with Model Integration

In [ ]:
# Gradio App Code

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import gradio as gr
import random

# Simulated models (replace with real models in production)
feature_names = ["Growing Stress", "Mood Swings", "Social Weakness"]
lr_coef = np.array([0.4, 0.3, 0.3])
tt_coef = np.array([0.35, 0.25, 0.4])

# Simulate dynamic probabilities
def simulate_proba(inputs):
    total = sum(inputs) + 0.01
    prob_low = max(0.0, 1 - total / 12)
    prob_high = min(1.0, total / 12)
    prob_med = 1 - prob_low - prob_high
    return [round(prob_low, 2), round(prob_med, 2), round(prob_high, 2)]

# Plot bar chart as image string
def plot_contributions(contribs, feature_names):
    fig, ax = plt.subplots()
    colors = []
    for c in contribs:
        if c >= 0.5:
            colors.append("red")
        elif c >= 0.25:
            colors.append("orange")
        else:
            colors.append("green")
    bars = ax.bar(feature_names, contribs, color=colors)
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.2f}', ha='center', fontsize=10)
    ax.set_ylabel("Contribution Score")
    ax.set_title("Risk Contribution by Factor")
    plt.ylim(0, 1)

    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    img_base64 = base64.b64encode(buf.getvalue()).decode()
    plt.close(fig)
    return f"<img src='data:image/png;base64,{img_base64}'/>"

# Main prediction function
def predict_risk_final(stress, mood, social, model_choice):
    inputs = np.array([stress, mood, social])

    if model_choice == "Logistic Regression":
        contrib_raw = inputs * lr_coef
    else:
        contrib_raw = inputs * tt_coef

    contribs = contrib_raw / (np.sum(contrib_raw) + 0.001)  # avoid zero division

    # Simulate dynamic probabilities
    probas = simulate_proba(inputs)
    pred = np.argmax(probas)

    # Risk labels and colors
    risk_labels = {0: ("Low Risk", "green"), 1: ("Moderate Risk", "orange"), 2: ("High Risk", "red")}
    label, color = risk_labels[pred]

    # Recommendations
    recs = {
        0: "Maintain healthy habits and monitor stress levels.",
        1: "Consider talking to a counselor. Monitor your mood and social interactions.",
        2: "Immediate consultation recommended. Prioritize stress management and seek support."
    }

    # Determine factor colors dynamically
    factor_colors = []
    for c in contribs:
        if c >= 0.5:
            factor_colors.append("red")
        elif c >= 0.25:
            factor_colors.append("orange")
        else:
            factor_colors.append("green")

    # Create output HTML
    result_html = f"""
    <div style='font-size:20px; font-weight:bold; color:{color};'>
        Predicted Class: {pred} ({label})<br>
        Confidence: {max(probas):.2f}<br><br>
    </div>
    <div style='font-size:16px;'>
        <b>Main Risk Factors:</b><br>
    """
    for i, f in enumerate(feature_names):
        result_html += f"<span style='color:{factor_colors[i]};'>- {f}: {contribs[i]*100:.1f}% contribution</span><br>"

    result_html += f"<br><b>Recommendation:</b> {recs[pred]}</div>"

    # Chart
    contrib_chart = plot_contributions(contribs, feature_names)

    return result_html + contrib_chart

In [ ]:
# Gradio Interface
iface = gr.Interface(
    fn=predict_risk_final,
    inputs=[
        gr.Number(label="Growing Stress", value=1),
        gr.Number(label="Mood Swings", value=1),
        gr.Number(label="Social Weakness", value=1),
        gr.Dropdown(["Logistic Regression", "Tabular Neural Network"], label="Model Selection")
    ],
    outputs=gr.HTML(),
    allow_flagging="never",
    title="Mental Health Risk Assessment Web App"
)

iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2fe27b9453114414b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
